In [2]:
# ===================================================================
# DAVI CA2: Educational Dataset - Complete Data Wrangling Notebook
# ===================================================================
# Authors: [Student Name 1] + [Student Name 2]
# Date: January 2026
# Purpose: Clean and prepare educational datasets for analysis
# ===================================================================

# ===================================================================
# CELL 1: Import Required Libraries
# ===================================================================
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

# For data visualization during cleaning
import matplotlib.pyplot as plt
import seaborn as sns

# Set display options
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)
pd.set_option('display.width', None)

print("✅ Libraries imported successfully")
print(f"Pandas version: {pd.__version__}")
print(f"NumPy version: {np.__version__}")

# ===================================================================
# CELL 2: Load All Datasets
# ===================================================================
print("\n" + "="*70)
print("LOADING ALL DATASETS")
print("="*70)

# Load datasets
try:
    df_course_codes = pd.read_csv('CA2 datasets & meta data/Course Codes.csv', encoding='utf-8')
    df_meta = pd.read_csv('CA2 datasets & meta data/Meta Data.csv', encoding='utf-8')
    df_profiles = pd.read_csv('CA2 datasets & meta data/Student Profiles.csv', encoding='utf-8')
    df_results = pd.read_csv('CA2 datasets & meta data/Student Results.csv', encoding='utf-8')
    df_survey = pd.read_csv('CA2 datasets & meta data/Student Survey.csv', encoding='utf-8')
    
    print("✅ All datasets loaded successfully\n")
    
    # Display shapes
    datasets = {
        'Course Codes': df_course_codes,
        'Meta Data': df_meta,
        'Student Profiles': df_profiles,
        'Student Results': df_results,
        'Student Survey': df_survey
    }
    
    for name, df in datasets.items():
        print(f"{name:20s}: {df.shape[0]:4d} rows × {df.shape[1]:2d} columns")
        
except FileNotFoundError as e:
    print(f"❌ Error: {e}")
    print("Please ensure all CSV files are in the same directory as this notebook")

# ===================================================================
# CELL 3: Initial Data Exploration - Course Codes
# ===================================================================
print("\n" + "="*70)
print("DATASET 1: COURSE CODES")
print("="*70)

print("\nFirst 5 rows:")
print(df_course_codes.head())

print("\nData Info:")
print(df_course_codes.info())

print("\nMissing Values:")
print(df_course_codes.isnull().sum())

print("\nUnique Courses:")
print(df_course_codes['COURSE NAME'].tolist())

# ===================================================================
# CELL 4: Initial Data Exploration - Student Profiles
# ===================================================================
print("\n" + "="*70)
print("DATASET 2: STUDENT PROFILES")
print("="*70)

print("\nFirst 5 rows:")
print(df_profiles.head())

print("\nData Info:")
print(df_profiles.info())

print("\nMissing Values Summary:")
missing_profiles = df_profiles.isnull().sum()
missing_pct = (missing_profiles / len(df_profiles) * 100).round(2)
missing_df = pd.DataFrame({
    'Missing Count': missing_profiles,
    'Percentage': missing_pct
}).sort_values('Percentage', ascending=False)
print(missing_df[missing_df['Missing Count'] > 0])

print("\nColumn Names:")
for i, col in enumerate(df_profiles.columns, 1):
    print(f"{i:2d}. {col}")

# Sample STUDENT IDs to understand format
print("\nSample STUDENT IDs (to understand format):")
print(df_profiles['STUDENT ID'].head(10).tolist())

# ===================================================================
# CELL 5: Initial Data Exploration - Student Results
# ===================================================================
print("\n" + "="*70)
print("DATASET 3: STUDENT RESULTS")
print("="*70)

print("\nFirst 10 rows:")
print(df_results.head(10))

print("\nData Info:")
print(df_results.info())

print("\nMissing Values:")
print(df_results.isnull().sum())

print("\nBasic Statistics:")
print(df_results.describe())

print("\nUnique values per column:")
for col in df_results.columns:
    print(f"{col:15s}: {df_results[col].nunique():4d} unique values")

print("\nPERIOD values:")
print(df_results['PERIOD'].value_counts().sort_index())

# ===================================================================
# CELL 6: Initial Data Exploration - Student Survey
# ===================================================================
print("\n" + "="*70)
print("DATASET 4: STUDENT SURVEY")
print("="*70)

print("\nFirst 10 rows:")
print(df_survey.head(10))

print("\nData Info:")
print(df_survey.info())

print("\nMissing Values:")
print(df_survey.isnull().sum())

print("\nBasic Statistics:")
print(df_survey.describe())

print("\nUnique values per column:")
for col in df_survey.columns:
    print(f"{col:20s}: {df_survey[col].nunique():4d} unique values")

print("\nPERIOD values:")
print(df_survey['PERIOD'].value_counts().sort_index())

# ===================================================================
# CELL 7: Parse STUDENT ID to Extract CLASS Code
# ===================================================================
print("\n" + "="*70)
print("PARSING STUDENT ID - EXTRACTING CLASS CODE")
print("="*70)

# STUDENT ID Format: XXXX-CCC/III
# XXXX = 4 digit number
# CCC = 3 digit CLASS code (this is what we need!)
# III = 3 digit student index within class

def extract_class_from_student_id(student_id):
    """
    Extract the CLASS code from STUDENT ID
    Format: XXXX-CCC/III 
    We need XXXX-CCC (first 4 digits + dash + 3 digit class code)
    Example: 1101-009/002 → class code is '1101-009'
    """
    try:
        if pd.isna(student_id):
            return None
        student_id = str(student_id).strip()
        
        # Split by '/' to remove the student index
        if '/' in student_id:
            class_code = student_id.split('/')[0]  # Gets '1101-009'
            return class_code
        else:
            return student_id if '-' in student_id else None
    except:
        return None

# Apply to all datasets that have STUDENT ID
print("\nExtracting CLASS codes from STUDENT ID...")

# Profiles
df_profiles['CLASS'] = df_profiles['STUDENT ID'].apply(extract_class_from_student_id)
print(f"✅ Profiles: Extracted CLASS for {df_profiles['CLASS'].notna().sum()} students")

# Results  
df_results['CLASS'] = df_results['STUDENT ID'].apply(extract_class_from_student_id)
print(f"✅ Results: Extracted CLASS for {df_results['CLASS'].notna().sum()} records")

# Survey
df_survey['CLASS'] = df_survey['STUDENT ID'].apply(extract_class_from_student_id)
print(f"✅ Survey: Extracted CLASS for {df_survey['CLASS'].notna().sum()} records")

# Show unique classes
print(f"\n📊 Unique CLASS codes found:")
all_classes = pd.concat([df_profiles['CLASS'], df_results['CLASS'], df_survey['CLASS']]).unique()
all_classes = sorted([c for c in all_classes if pd.notna(c)])
print(f"Total unique classes: {len(all_classes)}")
print(f"Classes: {all_classes}")

# Show distribution
print(f"\nCLASS distribution in Profiles:")
print(df_profiles['CLASS'].value_counts().sort_index())

# ===================================================================
# CELL 8: Cross-Dataset Relationship Analysis
# ===================================================================
print("\n" + "="*70)
print("CROSS-DATASET RELATIONSHIP ANALYSIS")
print("="*70)

# Check STUDENT ID consistency
unique_profiles = set(df_profiles['STUDENT ID'].unique())
unique_results = set(df_results['STUDENT ID'].unique())
unique_survey = set(df_survey['STUDENT ID'].unique())

print(f"\nUnique Student IDs:")
print(f"  Profiles: {len(unique_profiles)}")
print(f"  Results:  {len(unique_results)}")
print(f"  Survey:   {len(unique_survey)}")

print(f"\nStudents in Results but NOT in Profiles: {len(unique_results - unique_profiles)}")
print(f"Students in Survey but NOT in Profiles:  {len(unique_survey - unique_profiles)}")
print(f"Students in Profiles but NOT in Results: {len(unique_profiles - unique_results)}")
print(f"Students in Profiles but NOT in Survey:  {len(unique_profiles - unique_survey)}")

# Check PERIOD consistency
print(f"\nPERIOD Analysis:")
print(f"  Results PERIOD values: {sorted(df_results['PERIOD'].unique())}")
print(f"  Survey PERIOD values:  {sorted(df_survey['PERIOD'].unique())}")

# Check for duplicates
print(f"\nDuplicate Analysis:")
print(f"  Profiles duplicates (STUDENT ID): {df_profiles['STUDENT ID'].duplicated().sum()}")
print(f"  Results duplicates (STUDENT ID + PERIOD): {df_results.duplicated(subset=['STUDENT ID', 'PERIOD']).sum()}")
print(f"  Survey duplicates (STUDENT ID + PERIOD): {df_survey.duplicated(subset=['STUDENT ID', 'PERIOD']).sum()}")

# ===================================================================
# CELL 9: Clean Course Codes Dataset
# ===================================================================
print("\n" + "="*70)
print("CLEANING: COURSE CODES")
print("="*70)

df_course_codes_clean = df_course_codes.copy()

# Check for issues
print("Before cleaning:")
print(f"  Shape: {df_course_codes_clean.shape}")
print(f"  Nulls: {df_course_codes_clean.isnull().sum().sum()}")

# Remove any whitespace from column names
df_course_codes_clean.columns = df_course_codes_clean.columns.str.strip()

# Remove whitespace from string columns
for col in df_course_codes_clean.select_dtypes(include='object').columns:
    df_course_codes_clean[col] = df_course_codes_clean[col].str.strip()

# Check for duplicates
duplicates = df_course_codes_clean.duplicated().sum()
if duplicates > 0:
    print(f"  ⚠️  Found {duplicates} duplicate rows - removing...")
    df_course_codes_clean = df_course_codes_clean.drop_duplicates()

print("\nAfter cleaning:")
print(f"  Shape: {df_course_codes_clean.shape}")
print(df_course_codes_clean)

# ===================================================================
# CELL 10: Clean Student Profiles - Part 1 (Column Cleanup)
# ===================================================================
print("\n" + "="*70)
print("CLEANING: STUDENT PROFILES - PART 1 (Columns)")
print("="*70)

df_profiles_clean = df_profiles.copy()

print("Step 1: Standardize column names")
df_profiles_clean.columns = df_profiles_clean.columns.str.strip().str.upper()
print(f"  ✅ Standardized {len(df_profiles_clean.columns)} column names")

print("\nStep 2: Remove whitespace from all string columns")
for col in df_profiles_clean.select_dtypes(include='object').columns:
    if col != 'CLASS':  # Don't strip CLASS since we just created it
        df_profiles_clean[col] = df_profiles_clean[col].str.strip()
print("  ✅ Whitespace removed")

print("\nStep 3: Standardize and fix nationality columns")

# First, standardize the citizenship columns (Y/N instead of Yes/No)
print("\n📊 BEFORE Standardization:")
print(f"SG CITIZEN unique values: {df_profiles_clean['SG CITIZEN'].unique()}")
print(f"SG PR unique values: {df_profiles_clean['SG PR'].unique()}")
print(f"FOREIGNER unique values: {df_profiles_clean['FOREIGNER'].unique()}")

def standardize_yes_no(value):
    """
    Standardize Yes/No/Y/N values to consistent 'Y' or 'N'
    """
    if pd.isna(value):
        return 'N'
    
    value_str = str(value).strip().upper()
    
    if value_str in ['YES', 'Y', '1', 'TRUE']:
        return 'Y'
    elif value_str in ['NO', 'N', '0', 'FALSE', '']:
        return 'N'
    else:
        return 'N'

# Apply standardization
df_profiles_clean['SG CITIZEN'] = df_profiles_clean['SG CITIZEN'].apply(standardize_yes_no)
df_profiles_clean['SG PR'] = df_profiles_clean['SG PR'].apply(standardize_yes_no)
df_profiles_clean['FOREIGNER'] = df_profiles_clean['FOREIGNER'].apply(standardize_yes_no)

print("\n📊 AFTER Standardization:")
print(f"SG CITIZEN unique values: {df_profiles_clean['SG CITIZEN'].unique()}")
print(f"SG PR unique values: {df_profiles_clean['SG PR'].unique()}")
print(f"FOREIGNER unique values: {df_profiles_clean['FOREIGNER'].unique()}")

# Now create NATIONALITY_STATUS
def determine_nationality_simple(row):
    if row['SG CITIZEN'] == 'Y':
        return 'SG Citizen'
    elif row['SG PR'] == 'Y':
        return 'SG PR'
    elif row['FOREIGNER'] == 'Y':
        return 'Foreigner'
    else:
        return 'Unknown'

df_profiles_clean['NATIONALITY_STATUS'] = df_profiles_clean.apply(
    determine_nationality_simple, axis=1
)

print("\n✅ Created NATIONALITY_STATUS column")
print("\n📊 Distribution:")
nationality_counts = df_profiles_clean['NATIONALITY_STATUS'].value_counts()
print(nationality_counts)

print("\nPercentages:")
for status, count in nationality_counts.items():
    pct = (count / len(df_profiles_clean) * 100)
    print(f"  {status}: {count} ({pct:.1f}%)")

unknown_cases = df_profiles_clean[df_profiles_clean['NATIONALITY_STATUS'] == 'Unknown']
if len(unknown_cases) > 0:
    print(f"\n⚠️  WARNING: {len(unknown_cases)} students have 'Unknown' nationality")
    print("Sample:")
    print(unknown_cases[['STUDENT ID', 'SG CITIZEN', 'SG PR', 'FOREIGNER']].head(10))
else:
    print("\n✅ No Unknown nationality cases!")

print("\n🔍 Data Quality Check:")
df_profiles_clean['citizenship_flags_count'] = (
    (df_profiles_clean['SG CITIZEN'] == 'Y').astype(int) +
    (df_profiles_clean['SG PR'] == 'Y').astype(int) +
    (df_profiles_clean['FOREIGNER'] == 'Y').astype(int)
)

multiple_flags = df_profiles_clean[df_profiles_clean['citizenship_flags_count'] > 1]
if len(multiple_flags) > 0:
    print(f"  ⚠️  {len(multiple_flags)} students have multiple citizenship flags")
else:
    print("  ✅ No multiple flags")

no_flags = df_profiles_clean[df_profiles_clean['citizenship_flags_count'] == 0]
if len(no_flags) > 0:
    print(f"  ⚠️  {len(no_flags)} students have NO citizenship flag")
else:
    print("  ✅ All students have flags")

df_profiles_clean = df_profiles_clean.drop('citizenship_flags_count', axis=1)

# ===================================================================
# CELL 11: Clean Student Profiles - Part 2 (Date Columns)
# ===================================================================
print("\n" + "="*70)
print("CLEANING: STUDENT PROFILES - PART 2 (Dates)")
print("="*70)

# Handle date columns
date_columns = ['DOB', 'DATE ATTAINED HIGHEST QUALIFICATION', 
                'COMMENCEMENT DATE', 'COMPLETION DATE']

print("Converting date columns to datetime...")
for col in date_columns:
    print(f"\nProcessing: {col}")
    # Check current format
    print(f"  Sample values: {df_profiles_clean[col].head(3).tolist()}")
    
    # Try to convert to datetime
    df_profiles_clean[col] = pd.to_datetime(df_profiles_clean[col], errors='coerce')
    
    # Report conversion
    nulls = df_profiles_clean[col].isnull().sum()
    print(f"  ✅ Converted. Nulls: {nulls} ({nulls/len(df_profiles_clean)*100:.1f}%)")

# Calculate age from DOB
from datetime import datetime
current_date = pd.Timestamp('2026-01-19')  # Use assignment date

# ===================================================================
# FIX 1: Convert date columns to datetime format
# ===================================================================
print("\n🔧 Converting date columns to datetime format...")

df_profiles_clean['DOB'] = pd.to_datetime(df_profiles_clean['DOB'], errors='coerce')
df_profiles_clean['DATE ATTAINED HIGHEST QUALIFICATION'] = pd.to_datetime(df_profiles_clean['DATE ATTAINED HIGHEST QUALIFICATION'], errors='coerce')
df_profiles_clean['COMMENCEMENT DATE'] = pd.to_datetime(df_profiles_clean['COMMENCEMENT DATE'], errors='coerce')
df_profiles_clean['COMPLETION DATE'] = pd.to_datetime(df_profiles_clean['COMPLETION DATE'], errors='coerce')

print('✅ All date columns converted to datetime format')

df_profiles_clean['AGE'] = (current_date - df_profiles_clean['DOB']).dt.days / 365.25
df_profiles_clean['AGE'] = df_profiles_clean['AGE'].round(1)

print("\nAge Statistics:")
print(df_profiles_clean['AGE'].describe())

# Calculate course duration (will recalculate after filling dates)
df_profiles_clean['COURSE_DURATION_DAYS'] = (
    df_profiles_clean['COMPLETION DATE'] - df_profiles_clean['COMMENCEMENT DATE']
).dt.days

print("\nCourse Duration Statistics (before filling missing dates):")
print(df_profiles_clean['COURSE_DURATION_DAYS'].describe())

# ===================================================================
# CELL 12: Smart Date Filling Based on CLASS Code
# ===================================================================
print("\n" + "="*70)
print("SMART DATE FILLING - USING CLASS CODE")
print("="*70)

print("\n📊 BEFORE Date Filling:")
print(f"Missing COMMENCEMENT DATE: {df_profiles_clean['COMMENCEMENT DATE'].isnull().sum()} rows")
print(f"Missing COMPLETION DATE: {df_profiles_clean['COMPLETION DATE'].isnull().sum()} rows")

# Step 1: Create reference table of dates by CLASS
print("\n🔍 Step 1: Analyzing dates by CLASS (not PERIOD)...")

date_reference = df_profiles_clean.groupby('CLASS').agg({
    'COMMENCEMENT DATE': lambda x: x.mode()[0] if not x.mode().empty else pd.NaT,
    'COMPLETION DATE': lambda x: x.mode()[0] if not x.mode().empty else pd.NaT
}).reset_index()

date_reference.columns = ['CLASS', 'REFERENCE_COMMENCE', 'REFERENCE_COMPLETE']

print("\nDate Reference Table by CLASS:")
print(date_reference)

# Identify classes with no date info
classes_no_commence = date_reference[date_reference['REFERENCE_COMMENCE'].isnull()]['CLASS'].tolist()
classes_no_complete = date_reference[date_reference['REFERENCE_COMPLETE'].isnull()]['CLASS'].tolist()

if classes_no_commence:
    print(f"\n⚠️  WARNING: These CLASSes have NO commencement date info: {classes_no_commence}")
if classes_no_complete:
    print(f"⚠️  WARNING: These CLASSes have NO completion date info: {classes_no_complete}")

# Step 2: Fill missing dates
print("\n🔧 Step 2: Filling missing dates based on CLASS...")

# Merge reference dates
df_profiles_clean = df_profiles_clean.merge(
    date_reference,
    on='CLASS',
    how='left'
)

# Track what we fill
rows_filled_commence = 0
rows_filled_complete = 0

# Fill COMMENCEMENT DATE
mask_missing_commence = df_profiles_clean['COMMENCEMENT DATE'].isnull()
mask_has_ref_commence = df_profiles_clean['REFERENCE_COMMENCE'].notna()

df_profiles_clean.loc[mask_missing_commence & mask_has_ref_commence, 'COMMENCEMENT DATE'] = \
    df_profiles_clean.loc[mask_missing_commence & mask_has_ref_commence, 'REFERENCE_COMMENCE']

rows_filled_commence = (mask_missing_commence & mask_has_ref_commence).sum()

# Fill COMPLETION DATE
mask_missing_complete = df_profiles_clean['COMPLETION DATE'].isnull()
mask_has_ref_complete = df_profiles_clean['REFERENCE_COMPLETE'].notna()

df_profiles_clean.loc[mask_missing_complete & mask_has_ref_complete, 'COMPLETION DATE'] = \
    df_profiles_clean.loc[mask_missing_complete & mask_has_ref_complete, 'REFERENCE_COMPLETE']

rows_filled_complete = (mask_missing_complete & mask_has_ref_complete).sum()

print(f"\n✅ Filled {rows_filled_commence} missing COMMENCEMENT DATEs")
print(f"✅ Filled {rows_filled_complete} missing COMPLETION DATEs")

# Drop reference columns
df_profiles_clean = df_profiles_clean.drop(['REFERENCE_COMMENCE', 'REFERENCE_COMPLETE'], axis=1)

# Step 3: Recalculate duration
print("\n🔧 Step 3: Recalculating COURSE_DURATION_DAYS...")

df_profiles_clean['COURSE_DURATION_DAYS'] = (
    df_profiles_clean['COMPLETION DATE'] - df_profiles_clean['COMMENCEMENT DATE']
).dt.days

valid_durations = df_profiles_clean['COURSE_DURATION_DAYS'].notna().sum()
print(f"✅ Calculated duration for {valid_durations} rows")

# Final report
print("\n" + "="*70)
print("📊 AFTER Date Filling:")
print("="*70)
print(f"Missing COMMENCEMENT DATE: {df_profiles_clean['COMMENCEMENT DATE'].isnull().sum()} rows")
print(f"Missing COMPLETION DATE: {df_profiles_clean['COMPLETION DATE'].isnull().sum()} rows")
print(f"Missing COURSE_DURATION_DAYS: {df_profiles_clean['COURSE_DURATION_DAYS'].isnull().sum()} rows")

# Show still-missing cases
still_missing = df_profiles_clean[
    df_profiles_clean['COMMENCEMENT DATE'].isnull() | 
    df_profiles_clean['COMPLETION DATE'].isnull()
][['STUDENT ID', 'CLASS', 'COMMENCEMENT DATE', 'COMPLETION DATE']].copy()

if len(still_missing) > 0:
    print(f"\n⚠️  {len(still_missing)} rows still have missing dates:")
    print("\nBreakdown by CLASS:")
    print(still_missing['CLASS'].value_counts())
    print("\nThese CLASSes have no date information from any student.")
else:
    print("\n🎉 All dates successfully filled!")

print("\n📈 Course Duration Statistics:")
print(df_profiles_clean['COURSE_DURATION_DAYS'].describe())

# ===================================================================
# CELL 13: Clean Student Profiles - Part 3 (Missing Values)
# ===================================================================
print("\n" + "="*70)
print("CLEANING: STUDENT PROFILES - PART 3 (Missing Values)")
print("="*70)

print("\nMissing Values Summary:")
missing_summary = pd.DataFrame({
    'Column': df_profiles_clean.columns,
    'Missing_Count': df_profiles_clean.isnull().sum().values,
    'Missing_Pct': (df_profiles_clean.isnull().sum().values / len(df_profiles_clean) * 100).round(2)
})
missing_summary = missing_summary[missing_summary['Missing_Count'] > 0].sort_values('Missing_Pct', ascending=False)
print(missing_summary.to_string(index=False))

# Handle specific missing values
print("\nHandling missing values:")

# GENDER
print(f"\n  GENDER missing: {df_profiles_clean['GENDER'].isnull().sum()}")
if df_profiles_clean['GENDER'].isnull().sum() > 0:
    df_profiles_clean['GENDER'].fillna('Unknown', inplace=True)
    print("    ✅ Filled with 'Unknown'")

print("\nDecision: Keep other nulls as-is, document in presentation")

# ===================================================================
# CELL 14: Clean Student Results
# ===================================================================
print("\n" + "="*70)
print("CLEANING: STUDENT RESULTS")
print("="*70)

df_results_clean = df_results.copy()

print("Step 1: Standardize column names")
df_results_clean.columns = df_results_clean.columns.str.strip().str.upper()

print("\nStep 2: Check and clean STUDENT ID")
df_results_clean['STUDENT ID'] = df_results_clean['STUDENT ID'].str.strip()

print("\nStep 3: Clean and standardize PERIOD column")

def standardize_period(period_str):
    """Standardize PERIOD: 'Semester' → 'Sem'"""
    if pd.isna(period_str):
        return period_str
    period_str = str(period_str).strip()
    period_str = period_str.replace('Semester', 'Sem')
    period_str = period_str.replace('semester', 'Sem')
    period_str = period_str.replace('SEMESTER', 'Sem')
    period_str = ' '.join(period_str.split())
    return period_str

print(f"  BEFORE: {df_results_clean['PERIOD'].unique()}")
df_results_clean['PERIOD'] = df_results_clean['PERIOD'].str.strip()

# ===================================================================
# FIX 3: Standardize PERIOD values
# ===================================================================
print("\n🔧 Standardizing PERIOD values in Results...")
before_period = df_results_clean["PERIOD"].value_counts().sort_index().to_dict()
print(f"Before: {before_period}")

df_results_clean["PERIOD"] = df_results_clean["PERIOD"].str.replace("Sem1", "Sem 1", regex=False)
df_results_clean["PERIOD"] = df_results_clean["PERIOD"].str.replace("Sem2", "Sem 2", regex=False)
df_results_clean["PERIOD"] = df_results_clean["PERIOD"].str.replace("Sem3", "Sem 3", regex=False)
df_results_clean["PERIOD"] = df_results_clean["PERIOD"].str.replace("Sem4", "Sem 4", regex=False)
df_results_clean["PERIOD"] = df_results_clean["PERIOD"].str.strip()

after_period = df_results_clean["PERIOD"].value_counts().sort_index().to_dict()
print(f"After: {after_period}")
print("✅ PERIOD values standardized in Results")

df_results_clean['PERIOD'] = df_results_clean['PERIOD'].apply(standardize_period)
print(f"  AFTER: {df_results_clean['PERIOD'].unique()}")
print(f"  ✅ PERIOD standardized")

print("\nStep 4: Validate GPA values")
print(f"  GPA range: {df_results_clean['GPA'].min():.2f} to {df_results_clean['GPA'].max():.2f}")

invalid_gpa = df_results_clean[(df_results_clean['GPA'] < 0) | (df_results_clean['GPA'] > 5)]
if len(invalid_gpa) > 0:
    print(f"  ⚠️  Found {len(invalid_gpa)} invalid GPA values:")
    print(invalid_gpa)
else:
    print("  ✅ All GPA values are valid")

null_gpa = df_results_clean['GPA'].isnull().sum()
print(f"  Null GPAs: {null_gpa}")

print("\nStep 5: Validate ATTENDANCE values")
print(f"  ATTENDANCE range: {df_results_clean['ATTENDANCE'].min()} to {df_results_clean['ATTENDANCE'].max()}")
print(f"  Null ATTENDANCE: {df_results_clean['ATTENDANCE'].isnull().sum()}")

invalid_attendance = df_results_clean[(df_results_clean['ATTENDANCE'] < 0) | (df_results_clean['ATTENDANCE'] > 100)]
if len(invalid_attendance) > 0:
    print(f"  ⚠️  Found {len(invalid_attendance)} invalid ATTENDANCE values")
    print(invalid_attendance)
else:
    print("  ✅ All ATTENDANCE values are valid")

print("\n🔍 Re-checking duplicates after PERIOD standardization...")
duplicates_after = df_results_clean.duplicated(subset=['STUDENT ID', 'PERIOD']).sum()
if duplicates_after > 0:
    print(f"  ⚠️  Found {duplicates_after} new duplicates after standardization")
    df_results_clean = df_results_clean.drop_duplicates(subset=['STUDENT ID', 'PERIOD'], keep='first')
    print(f"  ✅ Removed duplicates")

print("\nStep 6: Check for duplicate records")
duplicates = df_results_clean.duplicated(subset=['STUDENT ID', 'PERIOD']).sum()
print(f"  Duplicates (STUDENT ID + PERIOD): {duplicates}")
if duplicates > 0:
    print("  ⚠️  Removing duplicates...")
    df_results_clean = df_results_clean.drop_duplicates(subset=['STUDENT ID', 'PERIOD'], keep='first')
    print(f"  ✅ Removed {duplicates} duplicate records")

print(f"\nFinal shape: {df_results_clean.shape}")

# ===================================================================
# CELL 15: Clean Student Survey
# ===================================================================
print("\n" + "="*70)
print("CLEANING: STUDENT SURVEY")
print("="*70)

df_survey_clean = df_survey.copy()

print("Step 1: Standardize column names")
df_survey_clean.columns = df_survey_clean.columns.str.strip().str.upper()

print("\nStep 2: Clean STUDENT ID and PERIOD")

def standardize_period(period_str):
    """Standardize PERIOD: 'Semester' → 'Sem'"""
    if pd.isna(period_str):
        return period_str
    period_str = str(period_str).strip()
    period_str = period_str.replace('Semester', 'Sem')
    period_str = period_str.replace('semester', 'Sem')
    period_str = period_str.replace('SEMESTER', 'Sem')
    period_str = ' '.join(period_str.split())
    return period_str

df_survey_clean['STUDENT ID'] = df_survey_clean['STUDENT ID'].str.strip()

print(f"  BEFORE: {df_survey_clean['PERIOD'].unique()}")
df_survey_clean['PERIOD'] = df_survey_clean['PERIOD'].str.strip()

# ===================================================================
# FIX 4: Standardize PERIOD values
# ===================================================================
print("\n🔧 Standardizing PERIOD values in Survey...")
before_period = df_survey_clean["PERIOD"].value_counts().sort_index().to_dict()
print(f"Before: {before_period}")

df_survey_clean["PERIOD"] = df_survey_clean["PERIOD"].str.replace("Sem1", "Sem 1", regex=False)
df_survey_clean["PERIOD"] = df_survey_clean["PERIOD"].str.replace("Sem2", "Sem 2", regex=False)
df_survey_clean["PERIOD"] = df_survey_clean["PERIOD"].str.replace("Sem3", "Sem 3", regex=False)
df_survey_clean["PERIOD"] = df_survey_clean["PERIOD"].str.replace("Sem4", "Sem 4", regex=False)
df_survey_clean["PERIOD"] = df_survey_clean["PERIOD"].str.strip()

after_period = df_survey_clean["PERIOD"].value_counts().sort_index().to_dict()
print(f"After: {after_period}")
print("✅ PERIOD values standardized in Survey")

df_survey_clean['PERIOD'] = df_survey_clean['PERIOD'].apply(standardize_period)
print(f"  AFTER: {df_survey_clean['PERIOD'].unique()}")
print(f"  ✅ PERIOD standardized")

print("\nStep 3: Validate survey response columns")
survey_cols = ['PRIOR KNOWLEDGE', 'COURSE RELEVANCE', 'TEACHING SUPPORT', 
               'COMPANY SUPPORT', 'FAMILY SUPPORT', 'SELF-STUDY HRS']

for col in survey_cols:
    print(f"\n  {col}:")
    print(f"    Range: {df_survey_clean[col].min()} to {df_survey_clean[col].max()}")
    print(f"    Nulls: {df_survey_clean[col].isnull().sum()}")
    print(f"    Unique: {df_survey_clean[col].nunique()} values")

print("\n🔍 Re-checking duplicates after PERIOD standardization...")
duplicates_after = df_survey_clean.duplicated(subset=['STUDENT ID', 'PERIOD']).sum()
if duplicates_after > 0:
    print(f"  ⚠️  Found {duplicates_after} new duplicates after standardization")
    df_survey_clean = df_survey_clean.drop_duplicates(subset=['STUDENT ID', 'PERIOD'], keep='first')
    print(f"  ✅ Removed duplicates")

print("\nStep 4: Check for duplicate records")
duplicates = df_survey_clean.duplicated(subset=['STUDENT ID', 'PERIOD']).sum()
print(f"  Duplicates (STUDENT ID + PERIOD): {duplicates}")
if duplicates > 0:
    print("  ⚠️  Removing duplicates...")
    df_survey_clean = df_survey_clean.drop_duplicates(subset=['STUDENT ID', 'PERIOD'], keep='first')
    print(f"  ✅ Removed {duplicates} duplicate records")

print(f"\nFinal shape: {df_survey_clean.shape}")

# ===================================================================
# CELL 16: Cross-Dataset Validation
# ===================================================================
print("\n" + "="*70)
print("CROSS-DATASET VALIDATION")
print("="*70)

students_profiles = set(df_profiles_clean['STUDENT ID'].unique())
students_results = set(df_results_clean['STUDENT ID'].unique())
students_survey = set(df_survey_clean['STUDENT ID'].unique())

print("\nStudent ID Coverage:")
print(f"  Profiles: {len(students_profiles)} students")
print(f"  Results:  {len(students_results)} students")
print(f"  Survey:   {len(students_survey)} students")

orphan_results = students_results - students_profiles
if len(orphan_results) > 0:
    print(f"\n  ⚠️  {len(orphan_results)} students in Results without Profile:")
    print(f"    {list(orphan_results)[:10]}...")

orphan_survey = students_survey - students_profiles
if len(orphan_survey) > 0:
    print(f"\n  ⚠️  {len(orphan_survey)} students in Survey without Profile:")
    print(f"    {list(orphan_survey)[:10]}...")

print("\nPERIOD Consistency:")
periods_results = set(df_results_clean['PERIOD'].unique())
periods_survey = set(df_survey_clean['PERIOD'].unique())
print(f"  Results periods: {sorted(periods_results)}")
print(f"  Survey periods:  {sorted(periods_survey)}")

# ===================================================================
# CELL 17: Create Master Dataset
# ===================================================================
print("\n" + "="*70)
print("CREATING MASTER DATASET")
print("="*70)

print("\nMerging strategy:")
print("  1. Merge Results with Survey on STUDENT ID + PERIOD")
print("  2. Merge with Profiles on STUDENT ID")

# Merge Results + Survey
df_results_survey = pd.merge(
    df_results_clean,
    df_survey_clean,
    on=['STUDENT ID', 'PERIOD', 'CLASS'],
    how='outer',
    indicator=True
)

print(f"\nResults + Survey merge:")
print(f"  Both: {(df_results_survey['_merge'] == 'both').sum()}")
print(f"  Only Results: {(df_results_survey['_merge'] == 'left_only').sum()}")
print(f"  Only Survey: {(df_results_survey['_merge'] == 'right_only').sum()}")

df_results_survey = df_results_survey.drop('_merge', axis=1)

# Merge with Profiles
df_master = pd.merge(
    df_profiles_clean,
    df_results_survey,
    on=['STUDENT ID', 'CLASS'],
    how='left',
    suffixes=('_profile', '_course')
)

print(f"\nMaster dataset created:")
print(f"  Shape: {df_master.shape}")
print(f"  Students: {df_master['STUDENT ID'].nunique()}")

print("\nMaster dataset columns:")
for i, col in enumerate(df_master.columns, 1):
    print(f"  {i:2d}. {col}")

# ===================================================================
# CELL 18: Final Data Quality Report
# ===================================================================

# --- NEW STEP: Remove Duplicates from Master Dataset ---
print("Processing Master Dataset Deduplication...")
rows_before = df_master.shape[0]

# drop_duplicates() removes rows where all columns are identical
df_master = df_master.drop_duplicates() 

rows_removed = rows_before - df_master.shape[0]
print(f"Done. Removed {rows_removed:,} duplicate rows.")

# --- GENERATE REPORT ---
print("\n" + "="*70)
print("FINAL DATA QUALITY REPORT")
print("="*70)

print("\n📊 CLEANED DATASETS SUMMARY:\n")

# ===================================================================
# FIX 5: Remove duplicates from Master Dataset
# ===================================================================
print("\n🔧 Removing duplicates from Master Dataset...")
before_dedup = len(df_master)
dup_count = df_master.duplicated(subset=["STUDENT ID", "PERIOD"]).sum()
print(f"Duplicates found: {dup_count}")

if dup_count > 0:
    dup_records = df_master[df_master.duplicated(subset=["STUDENT ID", "PERIOD"], keep=False)]
    print("Showing duplicate records:")
    print(dup_records[["STUDENT ID", "PERIOD", "GPA", "ATTENDANCE"]].sort_values(["STUDENT ID", "PERIOD"]).head(10))
    
    # Remove duplicates
    df_master = df_master.drop_duplicates(subset=["STUDENT ID", "PERIOD"], keep="first")
    after_dedup = len(df_master)
    removed = before_dedup - after_dedup
    print(f"✅ Removed {removed} duplicate rows")
    print(f"New shape: {df_master.shape}")
else:
    print("✅ No duplicates found")

summaries = {
    'Course Codes': df_course_codes_clean,
    'Student Profiles': df_profiles_clean,
    'Student Results': df_results_clean,
    'Student Survey': df_survey_clean,
    'Master Dataset': df_master  # This now contains the deduplicated data
}

for name, df in summaries.items():
    print(f"\n{name}:")
    print(f"  Rows: {df.shape[0]:,}")
    print(f"  Columns: {df.shape[1]}")
    print(f"  Total Nulls: {df.isnull().sum().sum():,}")
    print(f"  Memory: {df.memory_usage(deep=True).sum() / 1024**2:.2f} MB")
# ===================================================================
# CELL 19: Export Cleaned Datasets
# ===================================================================
print("\n" + "="*70)
print("EXPORTING CLEANED DATASETS")
print("="*70)

# Create directory
import os
os.makedirs('cleaned_data', exist_ok=True)

# Export
df_course_codes_clean.to_csv('cleaned_data/course_codes_clean.csv', index=False)
df_profiles_clean.to_csv('cleaned_data/student_profiles_clean.csv', index=False)
df_results_clean.to_csv('cleaned_data/student_results_clean.csv', index=False)
df_survey_clean.to_csv('cleaned_data/student_survey_clean.csv', index=False)
df_master.to_csv('cleaned_data/master_dataset.csv', index=False)

print("\n✅ All cleaned datasets exported to 'cleaned_data/' folder")

# ===================================================================
# CELL 20: Create Data Wrangling Documentation
# ===================================================================
print("\n" + "="*70)
print("DATA WRANGLING DOCUMENTATION (for PowerPoint)")
print("="*70)

wrangling_log = pd.DataFrame({
    'Field Name': [
        'All Columns (All Datasets)',
        'STUDENT ID (All Datasets)',
        'CLASS (Extracted from STUDENT ID)',
        'PERIOD (Results, Survey)',
        'NATIONALITY columns (Profiles)',
        'Date columns (Profiles)',
        'COMMENCEMENT DATE (Profiles)',
        'COMPLETION DATE (Profiles)',
        'COURSE_DURATION_DAYS (Profiles)',
        'GPA (Results)',
        'ATTENDANCE (Results)',
        'Survey responses (Survey)',
        'Duplicate records (Results)',
        'Duplicate records (Survey)',
        'GENDER (Profiles)',
    ],
    'Records Affected': [
        'All',
        f'{len(df_profiles_clean)} + {len(df_results_clean)} + {len(df_survey_clean)}',
        f'{len(df_profiles_clean)} + {len(df_results_clean)} + {len(df_survey_clean)}',
        f'{len(df_results_clean)} + {len(df_survey_clean)}',
        f'{len(df_profiles_clean)}',
        f'{len(df_profiles_clean)}',
        f'{rows_filled_commence}',
        f'{rows_filled_complete}',
        f'{valid_durations}',
        f'{len(df_results_clean)}',
        f'{len(df_results_clean)}',
        f'{len(df_survey_clean)}',
        f'{df_results.duplicated(subset=["STUDENT ID", "PERIOD"]).sum()}',
        f'{df_survey.duplicated(subset=["STUDENT ID", "PERIOD"]).sum()}',
        f'{df_profiles["GENDER"].isnull().sum()}',
    ],
    'Action Taken': [
        'Removed leading/trailing whitespace, standardized column names to uppercase',
        'Trimmed whitespace, validated format consistency across datasets',
        'Extracted 3-digit CLASS code from STUDENT ID format (XXXX-CCC/III)',
        'Trimmed whitespace, ensured consistent formatting',
        'Created NATIONALITY_STATUS column from SG CITIZEN, SG PR, FOREIGNER flags',
        'Converted to datetime format, created AGE and COURSE_DURATION_DAYS',
        'Filled missing dates using mode from same CLASS code',
        'Filled missing dates using mode from same CLASS code',
        'Calculated/recalculated after filling commence and completion dates',
        'Validated range (0-5), checked for nulls and outliers',
        'Validated range (0-100), checked for nulls and invalid values',
        'Validated response scales, checked for nulls and outliers',
        'Removed duplicate STUDENT ID + PERIOD combinations, kept first',
        'Removed duplicate STUDENT ID + PERIOD combinations, kept first',
        'Filled missing values with "Unknown"',
    ]
})

print("\n" + wrangling_log.to_string(index=False))

wrangling_log.to_csv('cleaned_data/data_wrangling_log.csv', index=False)
print("\n✅ Data wrangling log exported")

# ===================================================================
# CELL 21: Quick EDA
# ===================================================================
print("\n" + "="*70)
print("QUICK EXPLORATORY DATA ANALYSIS")
print("="*70)

print("\n1. GENDER Distribution:")
print(df_profiles_clean['GENDER'].value_counts())

print("\n2. NATIONALITY_STATUS Distribution:")
print(df_profiles_clean['NATIONALITY_STATUS'].value_counts())

print("\n3. CLASS Distribution:")
print(df_profiles_clean['CLASS'].value_counts().sort_index())

print("\n4. FULL-TIME OR PART-TIME Distribution:")
print(df_profiles_clean['FULL-TIME OR PART-TIME'].value_counts())

print("\n5. GPA Distribution by Period:")
print(df_results_clean.groupby('PERIOD')['GPA'].describe())

print("\n6. Average Survey Scores:")
survey_cols = ['PRIOR KNOWLEDGE', 'COURSE RELEVANCE', 'TEACHING SUPPORT', 
               'COMPANY SUPPORT', 'FAMILY SUPPORT']
print(df_survey_clean[survey_cols].mean().round(2))

print("\n7. Self-Study Hours Distribution:")
print(df_survey_clean['SELF-STUDY HRS'].describe())

print("\n" + "="*70)
print("✅ DATA WRANGLING COMPLETE!")
print("="*70)
print("\nNext Steps:")
print("  1. Review cleaned datasets in 'cleaned_data/' folder")
print("  2. Use master_dataset.csv for integrated analysis")
print("  3. Use individual clean datasets for specific analyses")
print("  4. Proceed to visualization phase (Plotly charts)")
print("="*70)


✅ Libraries imported successfully
Pandas version: 2.2.2
NumPy version: 1.26.4

LOADING ALL DATASETS
✅ All datasets loaded successfully

Course Codes        :    7 rows ×  2 columns
Meta Data           :   40 rows ×  3 columns
Student Profiles    :  307 rows × 15 columns
Student Results     :  555 rows ×  4 columns
Student Survey      :  543 rows ×  8 columns

DATASET 1: COURSE CODES

First 5 rows:
   CODE                        COURSE NAME
0  1101  Diploma in Data Analytics with AI
1  1102     Diploma in Business Management
2  2101   Certificate in Emerging Business
3  2102   Certificate in Talent Management
4  2013  Certificate in Data Visualization

Data Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7 entries, 0 to 6
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   CODE         7 non-null      int64 
 1   COURSE NAME  7 non-null      object
dtypes: int64(1), object(1)
memory usage: 244.0+ bytes
None

Miss